In [1]:
import argparse
import logging
from pathlib import Path
from typing import Literal, Optional

import numpy as np
import pandas as pd
from utils import (
    calculate_weights,
    get_available_trackers,
    load_trackers,
    get_rebalance_dates,
    cap_long_only_weights,
)
from backtest import backtest2
from entities import FX_TRACKER_DICT, EM_CDS_TRACKER_DICT

from bwlogger import StyleAdapter, basic_setup
from bwutils import open_file
from portfolio.construction import calculate_weights as calculate_weights_fh

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
dict_backtest = {}

In [ ]:
EM_CDS_TRACKER_DICT = {
    "AED": "GSCDABBE Index", # no FX Tracker
    "ARS": "GSCDARBE Index",
    "BRL": "GSCDBRBE Index",
    "CLP": "GSCDCLBE Index",
    "CNY": "GSCDCHBE Index",
    "COP": "GSCDCOBE Index",
    "IDR": "GSCDINBE Index",
    "MYR": "GSCDMABE Index",
    "MXN": "GSCDMEBE Index",
    "PAB": "GSCDPABE Index",
    "PEN": "GSCDPEBE Index",
    "PHP": "GSCDPHBE Index",
    "QAR": "GSCDQABE Index",
    "RUB": "GSCDRUBE Index",
    "SAR": "GSCDSABE Index",
    "ZAR": "GSCDSOBE Index",
    "TRY": "GSCDTUBE Index",
    "UAH": "GSCDUKBE Index",
}
FX_TRACKER_DICT = {
    "ARS": "JPFCTARS Index",
    "BRL": "JPFCTBRL Index",
    "CLP": "JPFCTCLP Index",
    "CNY": "JPFCTCNY Index",
    "COP": "JPFCTCOP Index",
    "CZK": "JPFCTCZK Index",  # no CDS tracker
    "HUF": "JPFCTHUF Index",  # no CDS tracker
    "IDR": "JPFCTIDR Index",
    "INR": "JPFCTINR Index",  # no CDS tracker
    "MXN": "JPFCTMXN Index",
    "MYR": "JPFCTMYR Index",
    "PAB": "JPFCTPAB Index",  # pegged to dollar?
    "PEN": "JPFCTPEN Index",
    "PHP": "JPFCTPHP Index",
    "PLN": "JPFCTPLN Index", # no CDS tracker
    "QAR": "JPFCTQAR Index",
    "RON": "JPFCTRON Index",  # no CDS tracker
    "RUB": "JPFCTRUB Index",
    "SAR": "JPFCTSAR Index",  # pegged to dollar?
    "SGD": "JPFCTSGD Index",  # no CDS tracker
    "THB": "JPFCTTHB Index",  # no CDS tracker
    "TRY": "JPFCTTRY Index",
    "TWD": "JPFCTTWD Index",  # no CDS tracker
    "UAH": "JPFCTUAH Index",
    "ZAR": "JPFCTZAR Index",
}



In [2]:
df_fx = load_trackers(FX_TRACKER_DICT)
df_cds = load_trackers(EM_CDS_TRACKER_DICT)
new_index = df_fx.index.union(df_cds.index).sort_values()
df_fx = df_fx.reindex(index=new_index, method="ffill").dropna(how="all")
df_cds = df_cds.reindex(index=new_index, method="ffill").dropna(how="all")

In [ ]:
CDS_FX_PAIRS = df_fx.columns.intersection(df_cds.columns).tolist()
CDS_FX_PAIRS

In [ ]:
dict_backtest["cdx_em"] = load_trackers({"CDX EM": "EREM5LD5 Index"})
dict_backtest["bench_fx_full_ivp_cap_1/3"] = backtest2(
    df_fx, method_weights="ivp", cap=1 / 3, vol_target=0.1
)[0]["assets"]
# df_backtest.to_excel(
#     OUTPUT_FOLDER.joinpath("portfolio_currencies.xlsx"), index_label="Date"
# )
dict_backtest["bench_cds_full_ivp_cap_1/3"] = backtest2(
    df_cds, method_weights="ivp", cap=1 / 3, vol_target=0.1
)[0]["assets"]
dict_backtest["bench_fx_w_pair_ivp_cap_1/3"] = backtest2(
    df_fx[CDS_FX_PAIRS], method_weights="ivp", cap=1 / 3, vol_target=0.1
)[0]["assets"]
# df_backtest.to_excel(
#     OUTPUT_FOLDER.joinpath("portfolio_currencies.xlsx"), index_label="Date"
# )
dict_backtest["bench_cds_w_pair_ivp_cap_1/3"] = backtest2(
    df_cds[CDS_FX_PAIRS], method_weights="ivp", cap=1 / 3, vol_target=0.1
)[0]["assets"]

In [ ]:
def backtest_long_cds_short_fx_pairs_eqw(df_fx, df_cds, inverted=False):
    name = "long_fx_short_cds" if inverted else "long_cds_short_fx"
    temp_dict_backtest = {}
    for ccy in CDS_FX_PAIRS:
        if ccy in ["CNY", "MYR", "RUB"]:  # not enough data
            continue
        s_fx = df_fx[ccy].copy().dropna()
        s_fx.name = s_fx.name + "_fx"
        s_cds = df_cds[ccy].copy().dropna()  # long CDS(sell protection)
        s_cds.name = s_cds.name + "_cds"
        if inverted:
            s_cds = s_cds.iloc[0] * s_cds / 100
        else:
            s_fx = s_fx.iloc[0] / s_fx * 100

        df_long_short = pd.concat([s_fx, s_cds], axis=1, join="inner").dropna()
        s = backtest2(df_long_short, method_weights="ivp", vol_target=0.1)[0]["assets"]
        s.name = f"{name}_{ccy.lower()}"
        temp_dict_backtest[s.name] = s

    tracker_df = pd.concat(temp_dict_backtest.values(), axis=1).copy().dropna(how="all")
    backtest = pd.Series(index=tracker_df.index, name=f"bench_{name}_eqw")
    backtest.iloc[0] = 100.0
    n_available = tracker_df.loc[backtest.index[0]].dropna().count()
    w = 1 / n_available
    q = backtest.iloc[0] * w / tracker_df.iloc[0]
    for t, tm1 in zip(backtest.index[1:], backtest.index[:-1]):
        pnl = ((tracker_df.loc[t] - tracker_df.loc[tm1]) * q).sum()
        backtest[t] = backtest[tm1] + pnl
        if t.month != tm1.month:
            n_available = tracker_df.loc[tm1].dropna().count()
            w = 1 / n_available
            q = backtest[tm1] * w / tracker_df.loc[tm1]
    temp_dict_backtest[f"bench_{name}_eqw"] = backtest
    return temp_dict_backtest

teste = backtest_long_cds_short_fx_pairs_eqw(df_fx, df_cds, inverted=True)
pd.concat(teste.values(), axis=1).to_clipboard()

In [ ]:
ccy = "BRL"
s_fx = df_fx[ccy].copy().dropna()
s_fx.name = s_fx.name + "_fx"
s_cds = df_cds[ccy].copy().dropna()  # long CDS(sell protection)
s_cds.name = s_cds.name + "_cds"
s_fx = s_fx.loc[s_cds.index.min() :]
s_fx = s_fx.iloc[0] / s_fx * 100

df_long_short = pd.concat([s_fx, s_cds], axis=1, join="inner").dropna()
backtest_teste, position_teste = backtest2(
    df_long_short, method_weights="ivp", vol_target=0.1
)
backtest_teste.plot()
backtest_teste.to_clipboard(excel=True)

In [ ]:
list_beta = []
for ccy in CDS_FX_PAIRS:
    index_intersect = (
        df_fx[ccy].dropna().index.intersection(df_cds[ccy].dropna().index)
    )[252*2:]
    corr = (
        np.log(df_fx[ccy])
        .diff(21)
        .ewm(halflife=252)
        .corr(np.log(df_cds[ccy]).diff(21))
        .loc[index_intersect]
    )
    beta = (
        corr
        * np.log(df_fx[ccy]).diff(21).ewm(halflife=252).std()
        / np.log(df_cds[ccy]).diff(21).ewm(halflife=252).std()
    )
    list_beta.append(beta)

df_betas = pd.concat(list_beta, axis=1)
df_betas.plot()

In [ ]:
list_beta = []
for ccy in CDS_FX_PAIRS:
    index_intersect = (
        df_fx[ccy].dropna().index.intersection(df_cds[ccy].dropna().index)
    )[252 * 2 :]
    corr = (
        np.log(df_fx[ccy])
        .diff(21)
        .expanding()
        .corr(np.log(df_cds[ccy]).diff(21))
        .loc[index_intersect]
    )
    beta = (
        corr
        * np.log(df_fx[ccy]).diff(21).expanding().std()
        / np.log(df_cds[ccy]).diff(21).expanding().std()
    )
    list_beta.append(beta)

df_betas = pd.concat(list_beta, axis=1)
df_betas.plot()

In [ ]:
df_betas